# 层和块

首先，我们回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.1721, -0.0884, -0.2530,  0.1242, -0.1654,  0.0171,  0.0757,  0.0934,
          0.1890, -0.0081],
        [ 0.0899, -0.0173, -0.1995,  0.0950, -0.1851,  0.0375,  0.0643, -0.0979,
          0.2362,  0.1151]], grad_fn=<AddmmBackward>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[-0.0327,  0.1899,  0.1142, -0.0694, -0.1873,  0.0234, -0.0245, -0.3187,
          0.0212, -0.0756],
        [-0.1519,  0.1842,  0.1986,  0.0640, -0.1525, -0.0355,  0.0439, -0.1693,
         -0.0252,  0.0356]], grad_fn=<AddmmBackward>)

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-7.2113e-02,  9.0570e-02,  1.7037e-01,  3.5454e-02, -1.6024e-01,
         -2.5255e-01, -1.1680e-02, -1.1580e-01,  3.2509e-01,  1.0342e-01],
        [-3.2259e-02,  2.0852e-02,  1.6723e-01,  5.4202e-02,  9.1880e-03,
         -3.5504e-01,  7.5906e-02, -1.0812e-01,  7.7832e-02,  1.1070e-04]],
       grad_fn=<AddmmBackward>)

在正向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.1721, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1010, grad_fn=<SumBackward0>)